# Gekko Xsimlab Core 2.0

Goals:
- instead of custom defaultdics, use native Gekko structure (since it is also just a python wrapper)
- get Latex output of Model
- try to get Environments to work, as attributes of self.m
    
Step1:
- wrap GEKKO class
- check all attribs
- create minimal model example
- run

# OK so what do I actually need to do?
FIRST OF ALL: build mini model test ground!
submodules of new setup: 1. state variables, 2. forcings, 3. intermediate

1. no need to unpack gk_SV + gk_Intermediates at finalize step
2. wrap all of the contextdicts (if necessary, why not lists?) into the model GEKKO class! (create subclass perhaps)

3. is there another way I can fix the "make_FX_flux" way of doing things? it is a bit clunky..
4. what about dimensionality??

So the idea of providing the mathematical equations in a submodule as python functions is very good! I should definitely do that!

2. provide generalistic interface, that can solve with GEKKO, but can also compute normally

In [1]:
from gekko import GEKKO
from collections import defaultdict
import numpy as np

from time import process_time
import xsimlab as xs

In [13]:
from enum import Enum
import itertools
import warnings

import attr
from attr._make import _CountingAttr

from xsimlab.utils import normalize_encoding
from xsimlab.variable import VarType, VarIntent, _as_dim_tuple, _as_group_tuple


def phydra_SV(
    dims=(),
    intent="in",
    group=None,
    groups=None,
    default=attr.NOTHING,
    validator=None,
    converter=None,
    static=False,
    description="",
    attrs=None,
    encoding=None,
):
    """Create a variable.
    Variables store useful metadata such as dimension labels, a short
    description, a default value, validators or custom,
    user-provided metadata.
    Variables are the primitives of the modeling framework, they
    define the interface of each process in a model.
    Variables should be declared exclusively as class attributes in
    process classes (i.e., classes decorated with :func:`process`).
    Parameters
    ----------
    dims : str or tuple or list, optional
        Dimension label(s) of the variable. An empty tuple
        corresponds to a scalar variable (default), a string or a 1-length
        tuple corresponds to a 1-d variable and a n-length tuple corresponds to
        a n-d variable. A list of str or tuple items may also be provided if
        the variable accepts different numbers of dimensions.
    intent : {'in', 'out', 'inout'}, optional
        Defines whether the variable is an input (i.e., the process needs the
        variable's value for its computation), an output (i.e., the process
        computes a value for the variable) or both an input/output (i.e., the
        process may update the value of the variable).
        (default: input).
    group : str, optional
        Variable group (depreciated, use ``groups`` instead).
    groups : str or list, optional
        Variable group(s).
    default : any, optional
        Single default value for the variable, ignored when ``intent='out'``
        (default: NOTHING). A default value may also be set using a decorator.
    validator : callable or list of callable, optional
        Function that could be called before or during a simulation (or when
        creating a new process instance) to check the value given
        for the variable.
        The function must accept three arguments:
        - the process instance (useful for accessing the value of other
          variables in that process)
        - the variable object (useful for accessing the variable metadata)
        - the value to be validated.
        The function should throw an exception in case where an invalid value
        is given.
        If a ``list`` is passed, its items are all are treated as validators.
        The validator can also be set using decorator notation.
    converter : callable, optional
        Callable that is called when setting a value for this variable, and that
        converts the value to the desired format. The callable must accept
        one argument and return one value. The value is converted before being
        passed to the validator, if any.
    static : bool, optional
        If True, the value of the (input) variable must be set once
        before the simulation starts and cannot be further updated
        externally (default: False). Note that it doesn't prevent updating
        the value internally, i.e., from within the process class in which
        the variable is declared if ``intent`` is set to 'out' or 'inout',
        or from another process class (foreign variable).
    description : str, optional
        Short description of the variable.
    attrs : dict, optional
        Dictionnary of additional metadata (e.g., standard_name,
        units, math_symbol...).
    encoding : dict, optional
        Dictionary specifying how to encode this variable's data into a
        serialized format (i.e., as a zarr dataset). Currently used keys
        include 'dtype', 'compressor', 'fill_value', 'order', 'filters'
        and 'object_codec'. See :func:`zarr.creation.create` for details
        about these options. Other keys are ignored.
    See Also
    --------
    :func:`attr.ib`
    :mod:`attr.validators`
    """
    metadata = {
        "var_type": VarType.VARIABLE,
        "dims": _as_dim_tuple(dims),
        "intent": VarIntent(intent),
        "groups": _as_group_tuple(groups, group),
        "static": static,
        "attrs": attrs or {},
        "description": description,
        "encoding": normalize_encoding(encoding),
    }

    if VarIntent(intent) == VarIntent.OUT:
        _init = False
        _repr = False
    else:
        _init = True
        _repr = True

    return attr.attrib(
        metadata=metadata,
        default=default,
        validator=validator,
        converter=converter,
        init=_init,
        repr=_repr,
        kw_only=True,
    )

In [16]:

test_1 = xs.variable()

In [17]:
test_2 = phydra_SV()

In [12]:
# Library of mathematical functions

def monod(Ressource, params={'halfsat':0.5}):
    return Ressource / (params['halfsat'] + Ressource)


In [13]:
x = GEKKO()

In [18]:
SV = x.SV(name='state variable 1')

halfsat = x.Param(name='halfsat param')

In [15]:
monod(1)

0.6666666666666666

In [23]:
x.Intermediate(monod(SV, params={'halfsat':halfsat}), name='p_nutlim')

0

In [37]:
SVarray = x.Array(x.SV,(2,2))
SVarray2 = x.Array(x.SV,(2,2))

In [38]:
SVarray[0,0].value #= 'hello'

0

In [44]:
type(SVarray[0,0])

gekko.gk_variable.GK_SV

In [31]:
monod(SVarray) * SVarray

array([[((((v12)/((0.5+v12))))*(v12)), ((((v13)/((0.5+v13))))*(v13))],
       [((((v14)/((0.5+v14))))*(v14)), ((((v15)/((0.5+v15))))*(v15))]],
      dtype=object)

In [32]:
SVarray * SVarray2

array([[((v12)*(v16)), ((v13)*(v17))],
       [((v14)*(v18)), ((v15)*(v19))]], dtype=object)

In [83]:
# Gekko Backend

In [12]:
@xs.process
class GekkoCore:
    m = xs.any_object()
    
    def initialize(self):
        print('initializing model core')
        self.m = GEKKO(remote=False, name='phydra')
        
    def finalize(self):
        print('finalizing gekko core: cleanup')
        #self.m.open_folder()
        self.m.cleanup()
        
        
@xs.process
class GekkoSolve:
    m = xs.foreign(GekkoCore, 'm')
    
    def run_step(self):
        print(self.m.__dict__)
        print('Running solver now')
        self.m.options.REDUCE = 3
        self.m.options.NODES = 3
        self.m.options.IMODE = 7
        
        solve_start = process_time()
        self.m.solve(disp=False)  # disp=True) # to print gekko output
        solve_end = process_time()

        print(f"ModelSolve done in {round(solve_end-solve_start,2)} seconds")
        
        print(self.m.__dict__)

In [13]:
@xs.process
class Time:
    m = xs.foreign(GekkoCore, 'm')
    
    days = xs.variable(dims='time', description='time in days')
    # for indexing xarray IO objects
    time = xs.index(dims='time', description='time in days')

    def initialize(self):
        print('Initializing Model Time')
        self.time = self.days

        # ASSIGN MODEL SOLVING TIME HERE:
        self.m.time = self.time
        
        # add variable keeping track of time within model:
        self.m.timevar = self.m.Var(0, lb=0)
        self.m.Equation(self.m.timevar.dt() == 1)


@xs.process
class Environment:
    m = xs.foreign(GekkoCore, 'm')
    
    def initialize(self):
        self.m.env = defaultdict(list)
        
    def run_step(self):
        self.m.env.default_factory = None
        print(self.m.env)

In [14]:
def monod(Ressource, params={'halfsat':0.5}):
    return Ressource / (params['halfsat'] + Ressource)

In [117]:
class GEKKO_phydra(GEKKO):
    """ This class inherits GEKKO, and adds additional functionality """
    def __init__(self):
        super(GEKKO_phydra,self).__init__()
        self.name = "PhydraCore"
    
    def initialize_array(self, gk_class, label, shape, initval = None):
        """This function returns a labeled multi-dimensional m.array of supplied base gk_class"""
        gk_array = self.Array(gk_class, shape)
        
        it = np.nditer(gk_array, flags=['refs_ok','multi_index'], op_flags=['readwrite'])
        i=0
        while not it.finished:
            gk_array[it.multi_index].__dict__['NAME'] = f"{label}_{str(i)}"
            
            if initval != None:
                gk_array[it.multi_index].VALUE = initval
            
            i += 1
            it.iternext()
        
        return gk_array

    pass

In [118]:
m = GEKKO_phydra()

In [119]:
m.initialize_array(m.SV, 'hello', (1))

array([0], dtype=object)

In [124]:
m.__dict__['_variables'][0].value

0

In [161]:
def monod(Input, Output, params={'halfsat':0.5}):
    return Input / (params['halfsat'] + Input) * Output

In [153]:
m.Param(1,name='halfsat')

1

In [156]:
m._parameters

['halfsat']

In [165]:
# can i create an environment? try this first!

monod(m.Array(m.SV,(2,2)), m.Array(m.SV,(2,2)))

array([[((((v22)/((0.5+v22))))*(v26)), ((((v23)/((0.5+v23))))*(v27))],
       [((((v24)/((0.5+v24))))*(v28)), ((((v25)/((0.5+v25))))*(v29))]],
      dtype=object)

In [148]:
# so! this abstract flow needs access to the actual model object!

# so that I can initialize it within the gekko m object (GEKKO_phydra)
# but hm.. so what I want is a flexible store of model functions, together with their inputs/outputs & pars

# so that I can initialize it: 
    #- either taking standard xs.variables as input
    #- or the Gekko params and inputs (which could blow up in dims)
# i think just gekko is fine for now!

class Flux:
    def __init__(self, function, inputs, outputs):
        self.function = function
        self.inputs = inputs
        self.outputs = outputs
        
    def initialize_GK(self):
        print(self.function)
        print(self.inputs)
        print(self.outputs)
        
    def run_func(self):
        return self.function(self.inputs, self.outputs)

In [149]:
Flux(monod, 'N', 'P').initialize_GK()

<function monod at 0x1153c0a60>
N
P


In [150]:
Flux(monod, 'N', 'P').run_func()

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [125]:
class Environment:
    """AIM:
    - create an abstract model setup, that can be set up in a desired number of dimensions
    - this should contain: 
        Statevariables with labels (1D)
        Forcing
        Fluxes
    """
    def __init__(self,m):
        # record the GEKKO instances where the objects are added
        self.m = m
        self.components = defaultdict(list)
        self.forcings = defaultdict(list)
        self.fluxes = defaultdict(list)
        
    def add_obj(self, name='',n=None):
        x = name.lower() + str(len(self.m._objects) + 1)
        if n==None:
            self.m._objects.append(x+'='+name)
        else:
            self.m._objects.append(x+'='+name+'('+str(n)+')')
        return x
    
        return gk_array

In [126]:
m.env = Environment(m)

In [127]:
m.env.__dict__

{'m': <__main__.GEKKO_phydra at 0x115d9eb50>,
 'components': defaultdict(list, {}),
 'forcings': defaultdict(list, {}),
 'fluxes': defaultdict(list, {})}

In [101]:
a = env.add_obj('Hello')

In [102]:
m.__dict__

{'_remote': True,
 '_server': 'http://byu.apmonitor.com',
 'options': <gekko.gk_global_options.GKGlobalOptions at 0x115d9aa60>,
 '_id': 3,
 '_gui_open': False,
 '_constants': [],
 '_parameters': [],
 '_variables': [],
 '_intermediates': [],
 '_inter_equations': [],
 '_equations': [],
 '_objectives': [],
 '_connections': [],
 '_objects': ['hello1=Hello'],
 '_compounds': [],
 '_raw': [],
 'time': None,
 '_model_initialized': False,
 '_csv_status': None,
 '_model': '',
 '_model_name': 'gk_model3',
 '_path': '/var/folders/s8/8ypwvww534g_q5s51__36bmc0000gp/T/tmp_wlkeo8cgk_model3',
 'path': '/var/folders/s8/8ypwvww534g_q5s51__36bmc0000gp/T/tmp_wlkeo8cgk_model3',
 '_extra_files': [],
 'solver_options': []}

In [90]:
x_array = initialize_array(m.SV, 'Nutrient', (3,3,3), initval=0)

In [163]:
def initialize_array(gk_class, label, shape, initval = None):
    """This function returns a labeled multi-dimensional m.array of supplied base gk_class"""
    gk_array = m.Array(gk_class, shape)
    
    it = np.nditer(gk_array, flags=['refs_ok','multi_index'], op_flags=['readwrite'])

    i=0
    while not it.finished:
        gk_array[it.multi_index].__dict__['NAME'] = f"{label}_{str(i)}"
        
        if initval != None:
            gk_array[it.multi_index].VALUE = initval
            
        i += 1
        #print(it.value)
        it.iternext()
        
    return gk_array

In [15]:
# wHAT IS THE JOB OF ENVIRONMENTS.. do i need them??
# How does dimensionality fit in there??
# i basically have 2 choices for dims, either keep all SVs as m.Array, and all computations that way
# or have component dim within env, and then duplicate into larger dim structure.. which would be kinda cool!
# can i do that??

#can i do this with a class like below?

class Flowsheet():        
    def __init__(self,m,stream_level=1):
        # record the GEKKO instances where the objects are added
        self.m = m
        self.sl = stream_level # use pressure and energy balance
        if self.sl>=1:
            # use t,p,h,ndot,x[i] with STREAM_LEVEL = 1
            # models such as flash or other VLE require this
            self.m.options.STREAM_LEVEL = 1
        else:
            # use ndot,x[i] with STREAM_LEVEL = 0
            # track compositions only for blending and transport calculations
            self.m.options.STREAM_LEVEL = 0
        return
        
    def add_obj(self,name='',n=None):
        x = name.lower() + str(len(self.m._objects) + 1)
        if n==None:
            self.m._objects.append(x+'='+name)
        else:
            self.m._objects.append(x+'='+name+'('+str(n)+')')
        return x

In [16]:
@xs.process
class Component:
    label = xs.variable(intent='out')
    value = xs.variable(intent='out', dims='time')
    SV = xs.variable(intent='out')
    
    m = xs.foreign(GekkoCore, 'm')
    
    def initialize(self):
        self.label = self.__xsimlab_name__
        print('initializing SV')
        self.SV = self.m.SV(name='Nutrient')
        self.value = self.SV.value
    
    def run_step(self):
        print('assembling Equations')
        #print(self.label, self.value)
        self.m.Equation(self.SV.dt() == sum([flux for flux in self.m.env[self.label]]))
        
    def finalize_step(self):
        pass

@xs.process
class Intermediate:
    label = xs.variable(intent='in')
    
    m = xs.foreign(GekkoCore, 'm')
    
    def initialize(self):
        print('initializing Intermediate')
        self.m.env[self.label].append(0.1)
        pass

In [17]:
[1,2]

[1, 2]

In [ ]:
# TODO:
# 1. add value storage, both in Env and Comp
# 2. add a simple flux
# 3. test dim init: within Comp 1D, within Env, flexible

#IDEA:

remove the ENVS for noW! none of my use cases actually employ it!
so it would be awkward to present it there like that!!!

instead, just basic 0D model, but clean interface should be the goal!

in effect, as long as I use m.Array instead of m.SV, I should be able to add any kind of dims later on!

In [255]:
import string
import random

def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

In [340]:
@xs.process
class GekkoCore:
    m = xs.any_object()
    
    def initialize(self):
        print('initializing model core')
        self.m = GEKKO(remote=False, name='phydra')
        
        # add defaultdict of list that dynamically stores fluxes by component label
        self.m.fluxes = defaultdict(list)
        
    def finalize(self):
        print('finalizing gekko core: cleanup')
        #self.m.open_folder()
        self.m.cleanup()

        
@xs.process
class GekkoContext:
    """ Inherited by all other model processes to access GekkoCore"""
    m = xs.foreign(GekkoCore, 'm')

@xs.process
class GekkoSolve(GekkoContext):
    
    def finalize_step(self):
        print(self.m.__dict__)
        print('Running solver now')
        self.m.options.REDUCE = 3
        self.m.options.NODES = 3
        self.m.options.IMODE = 7
        
        solve_start = process_time()
        self.m.solve(disp=False)  # disp=True) # to print gekko output
        solve_end = process_time()

        print(f"ModelSolve done in {round(solve_end-solve_start,2)} seconds")
        
        print(self.m.__dict__)
        
@xs.process
class Time(GekkoContext):
    
    days = xs.variable(dims='time', description='time in days')
    # for indexing xarray IO objects
    time = xs.index(dims='time', description='time in days')

    def initialize(self):
        print('Initializing Model Time')
        self.time = self.days

        # ASSIGN MODEL SOLVING TIME HERE:
        self.m.time = self.time
        
        # add variable keeping track of time within model:
        self.m.timevar = self.m.Var(0, lb=0)
        self.m.Equation(self.m.timevar.dt() == 1)

        
@xs.process
class StateVariable(GekkoContext):
    label = xs.variable(intent='out')
    
    value = xs.variable(intent='out', dims='time', description='stores the value of component state variable')
    SV = xs.any_object(description='stores the gekko variable')
    
    initVal = xs.variable(intent='in', description='initial value of component')
    
    def initialize(self):
        self.label = self.__xsimlab_name__
        print(f"component {self.label} is initialized")
        
        self.SV = self.m.SV(self.initVal, name=self.label)
        self.value = self.SV.value
    
    def run_step(self):
        print('assembling Equations')
        #print(self.label, self.value)
        print(self.m.fluxes)
        self.m.Equation(self.SV.dt() == sum([flux for flux in self.m.fluxes[self.label]]))
        
    def finalize_step(self):
        pass

def monod(Input, Output, params={'halfsat':0.5}):
    return Input / (params['halfsat'] + Input) * Output
# now try to use this above in the flux below, and add func group dim parameter handling, that's intuitive!

@xs.process
class Fluxes(GekkoContext):
    label = xs.variable(intent='out')
    
    components = xs.variable(intent='in', dims=id_generator())
    
    def initialize(self):
        self.label = self.__xsimlab_name__
        print(f"flux {self.label} for {self.components} is initialized")
        
        flux = self.m.Param(0.1)
        for component in self.components:
            self.m.fluxes[component].append(flux)

In [341]:
a = xs.Model({'core':GekkoCore, 'solver':GekkoSolve, 'time':Time, 'N':StateVariable, 'P':StateVariable, 'growth':Fluxes})

In [342]:
in_ds = xs.create_setup(model=a,
                        clocks={'clock': [0,1]},
                        input_vars={
                            'time__days': ('time', np.arange(0, 1, 0.1)),
                            
                            'N':{'initVal':1},
                            'P':{'initVal':1},
                            'growth':{'components':['N','P']}
                        
                        },
                        output_vars={
                            'N__value':None,
                            'P__value':None
                        
                        })

In [343]:
with a:
    out_ds = in_ds.xsimlab.run()

initializing model core
Initializing Model Time
component N is initialized
component P is initialized
flux growth for ['N' 'P'] is initialized
assembling Equations
defaultdict(<class 'list'>, {'N': [0.1], 'P': [0.1]})
assembling Equations
defaultdict(<class 'list'>, {'N': [0.1], 'P': [0.1]})
{'_remote': False, '_server': 'http://byu.apmonitor.com', 'options': <gekko.gk_global_options.GKGlobalOptions object at 0x11673cdf0>, '_id': 39, '_gui_open': False, '_constants': [], '_parameters': [0.1], '_variables': [0, 1, 1], '_intermediates': [], '_inter_equations': [], '_equations': [<gekko.gekko.EquationObj object at 0x1156d3fd0>, <gekko.gekko.EquationObj object at 0x115b0fee0>, <gekko.gekko.EquationObj object at 0x115f0e070>], '_objectives': [], '_connections': [], '_objects': [], '_compounds': [], '_raw': [], 'time': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]), '_model_initialized': False, '_csv_status': None, '_model': '', '_model_name': 'phydra', '_path': '/var/folders/s8/8

In [344]:
out_ds

<xarray.Dataset>
Dimensions:             (BHWE2T: 2, clock: 2, time: 10)
Coordinates:
  * clock               (clock) int64 0 1
  * time                (time) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9
Dimensions without coordinates: BHWE2T
Data variables:
    N__initVal          int64 1
    N__value            (time) float64 1.0 1.01 1.02 1.03 ... 1.07 1.08 1.09
    P__initVal          int64 1
    P__value            (time) float64 1.0 1.01 1.02 1.03 ... 1.07 1.08 1.09
    growth__components  (BHWE2T) <U1 'N' 'P'
    time__days          (time) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9

In [18]:
MDL = xs.Model({'core':GekkoCore, 'time':Time, 
                
                'env':Environment, 'SV':Component, 'Flux':Intermediate,
               
                'solve':GekkoSolve
               })

In [19]:
in_ds = xs.create_setup(model=MDL,
                        clocks={'clock': [0,1]},
                        input_vars={
                            'time':{'days':np.arange(1,10)},
                            'Flux__label':'SV'
                        },
                        output_vars={
                            'SV__value':None
                        })

In [20]:
with MDL:
    out_ds = in_ds.xsimlab.run()

initializing model core
Initializing Model Time
initializing SV
initializing Intermediate
defaultdict(None, {'SV': [0.1]})
assembling Equations
{'_remote': False, '_server': 'http://byu.apmonitor.com', 'options': <gekko.gk_global_options.GKGlobalOptions object at 0x1141f61f0>, '_id': 1, '_gui_open': False, '_constants': [], '_parameters': [], '_variables': [0, 0], '_intermediates': [], '_inter_equations': [], '_equations': [<gekko.gekko.EquationObj object at 0x1142373a0>, <gekko.gekko.EquationObj object at 0x114237fa0>], '_objectives': [], '_connections': [], '_objects': [], '_compounds': [], '_raw': [], 'time': array([1, 2, 3, 4, 5, 6, 7, 8, 9]), '_model_initialized': False, '_csv_status': None, '_model': '', '_model_name': 'phydra', '_path': '/var/folders/s8/8ypwvww534g_q5s51__36bmc0000gp/T/tmpxe8v5w_2phydra', 'path': '/var/folders/s8/8ypwvww534g_q5s51__36bmc0000gp/T/tmpxe8v5w_2phydra', '_extra_files': [], 'solver_options': [], 'timevar': 0, 'env': defaultdict(None, {'SV': [0.1]})}
R

In [11]:
out_ds

<xarray.Dataset>
Dimensions:      (clock: 2, time: 9)
Coordinates:
  * clock        (clock) int64 0 1
  * time         (time) int64 1 2 3 4 5 6 7 8 9
Data variables:
    Flux__label  <U2 'SV'
    SV__value    (time) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8
    time__days   (time) int64 1 2 3 4 5 6 7 8 9

In [63]:
print(x)
q_dx

growthrate


[1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4, 1.45]

In [58]:
q.Connection(q_x,x)

m.Connection(x,q_x)

In [53]:
q = GEKKO(remote=False)

In [62]:
m = GEKKO(remote=False)

In [55]:
q_x = q.Param(1, name='growthrate')

q_dx = q.SV(1, name='X')

In [56]:
q_dxdt = q.Equation(q_dx.dt()==q_x/2)

In [63]:
x = m.Param(1, name='growthrate')

dx = m.SV(1, name='X')

In [64]:
[par.name for par in m._parameters]

['growthrate']

In [65]:
[var.name for var in m._variables]

['x']

In [66]:
[var.value for var in m._variables]

[1]

In [67]:
dxdt = m.Equation(dx.dt()==x/2)

In [68]:
[eq.value for eq in m._equations]

['$x=((growthrate)/(2))']

In [69]:
dxdt.value

'$x=((growthrate)/(2))'

In [59]:
m.time=np.arange(0,1,.1)

m.options.IMODE=7

m.solve(disp=False)

Exception: @error: Degrees of Freedom
 * Error: DOF must be zero for this mode
 STOPPING...


In [60]:
q.time=np.arange(0,1,.1)

q.options.IMODE=7

q.solve(disp=False)

In [70]:
[type(var.value) for var in m._variables]

[gekko.gk_operators.GK_Value]

In [101]:
dx.value

1

In [12]:
m.__dict__
# dir(m)

{'_remote': False,
 '_server': 'http://byu.apmonitor.com',
 'options': <gekko.gk_global_options.GKGlobalOptions at 0x1048a36a0>,
 '_id': 0,
 '_gui_open': False,
 '_constants': [],
 '_parameters': [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]],
 '_variables': [[1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4, 1.45]],
 '_intermediates': [],
 '_inter_equations': [],
 '_equations': [<gekko.gekko.EquationObj at 0x1105f9700>],
 '_objectives': [],
 '_connections': [],
 '_objects': [],
 '_compounds': [],
 '_raw': [],
 'time': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
 '_model_initialized': True,
 '_csv_status': None,
 '_model': 'auto-generated',
 '_model_name': 'gk_model0',
 '_path': '/var/folders/s8/8ypwvww534g_q5s51__36bmc0000gp/T/tmp6zq07x1rgk_model0',
 'path': '/var/folders/s8/8ypwvww534g_q5s51__36bmc0000gp/T/tmp6zq07x1rgk_model0',
 '_extra_files': [],
 'solver_options': [],
 'csv_status': 'generated'}

In [1]:
import numpy as np
from gekko import GEKKO
import matplotlib.pyplot as plt

m = GEKKO(remote=False)    # create GEKKO model

halfsat_const = m.Param(0.5)#, name='k_N')
N0 = m.Param(1.)#, name='$N^\emptyset$')
inflow_rate = m.Param(0.1)#, name='f^\\emptyset')
mortality_rate = m.Param(0.1)#, name='m^P')

N = m.SV(1)#, name='N_1')
P = m.SV(0.1)#, name='P_1')

In [2]:
start_range = 5
start_tinysteps=np.arange(0,start_range, .1)
t = np.arange(start_range,10)

new_t = np.concatenate([start_tinysteps, t])
time = np.arange(0,1,.1) # new_t

nutlim = m.Intermediate(N/(N+halfsat_const)*P)#, name='\lambda^N')
N_influx = m.Intermediate(N0 * inflow_rate)#, name='influx^N')
mortality = m.Intermediate(P * mortality_rate)#, name='mort^P')

m.time = time

dNdt = m.Equation(N.dt()==N0 *inflow_rate - N/(N+halfsat_const)*P)
dPdt = m.Equation(P.dt()==N/(N+halfsat_const)*P - P * mortality_rate)

m.options.REDUCE = 3
m.options.NODES = 3
m.options.IMODE = 7
#m.options.DIAGLEVEL = 4

In [3]:
dNdt.name

AttributeError: 'EquationObj' object has no attribute 'name'

In [4]:
m.__dict__

{'_remote': False,
 '_server': 'http://byu.apmonitor.com',
 'options': <gekko.gk_global_options.GKGlobalOptions at 0x10336e280>,
 '_id': 0,
 '_gui_open': False,
 '_constants': [],
 '_parameters': [0.5, 1.0, 0.1, 0.1],
 '_variables': [1, 0.1],
 '_intermediates': [0, 0, 0],
 '_inter_equations': ['((((v1)/((v1+p1))))*(v2))',
  '((p2)*(p3))',
  '((v2)*(p4))'],
 '_equations': [<gekko.gekko.EquationObj at 0x11226b880>,
 '_objectives': [],
 '_connections': [],
 '_objects': [],
 '_compounds': [],
 '_raw': [],
 'time': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
 '_model_initialized': False,
 '_csv_status': None,
 '_model': '',
 '_model_name': 'gk_model0',
 '_path': '/var/folders/s8/8ypwvww534g_q5s51__36bmc0000gp/T/tmp93ipijvngk_model0',
 'path': '/var/folders/s8/8ypwvww534g_q5s51__36bmc0000gp/T/tmp93ipijvngk_model0',
 '_extra_files': [],
 'solver_options': []}

In [6]:
#m.gk_logic_tree()
#m.get_names()
m.open_folder()

m.cleanup()

In [10]:
m.solve()

 ----------------------------------------------------------------
 APMonitor, Version 0.9.2
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 Called files(          55 )
 File copy failed: overrides.dbs does not exist
 files: overrides.dbs does not exist
 ARGUMENT            1 : gk_model1
 ARGUMENT            2 : 
 
 Run id   : 2020y07m01d00h14m56.892s
 
 COMMAND LINE ARGUMENTS
 coldstart:            0
 imode    :            7
 dbs_read :  T
 dbs_write:  T
 specs    :  T
 
 sqs selected
 Called files(          35 )
 READ info FILE FOR VARIABLE DEFINITION: gk_model1.info
 Parsing model file gk_model1.apm
 Read model file (sec):   2.0000000076834112E-003
 Initialize constants (sec):   0.0000000000000000     
 Determine model size (sec):   9.9999998928979039E-004
 Allocate memory (sec):   0.0000000000000000     
 Parse and store model (sec):   0.0000000000000000     
 
 --------- APM Model Size ------------
 Each time step contains
   Objec

Exception: @error: Model Expression
 *** Error in syntax of function string: Invalid element: v1

Position: 5                   
 ((((v1)/((v1+p1))))*(v2))
     ?



In [4]:
print(N.name)
type(N.value)

n_1


gekko.gk_operators.GK_Value

In [5]:
m.open_folder()

In [5]:
P.name

'phytoplankton'

In [6]:
m.get_names()

Found __
Found halfsat_const
Found N0
Found inflow_rate
Found mortality_rate
Found N
Found P
Found nutlim
Found N_influx
Found mortality
Found _4


In [2]:
m = GEKKO(remote=False)

In [3]:
x = m.Param(1, name='growthrate')

dx = m.SV(1, name='X')

In [4]:
[par.name for par in m._parameters]

['growthrate']

In [5]:
[var.name for var in m._variables]

['x']

In [6]:
[var.value for var in m._variables]

[1]

In [7]:
dxdt = m.Equation(dx.dt()==x/2)

In [10]:
[eq.value for eq in m._equations]

['$x=((growthrate)/(2))']

In [8]:
dxdt.value

'$x=((growthrate)/(2))'

In [11]:
m.time=np.arange(0,1,.1)

m.options.IMODE=7

m.solve(disp=False)

In [12]:
[var.value for var in m._variables]

[[1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4, 1.45]]

In [13]:
dx.value

[1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4, 1.45]

In [12]:
m.__dict__
# dir(m)

{'_remote': False,
 '_server': 'http://byu.apmonitor.com',
 'options': <gekko.gk_global_options.GKGlobalOptions at 0x1048a36a0>,
 '_id': 0,
 '_gui_open': False,
 '_constants': [],
 '_parameters': [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]],
 '_variables': [[1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4, 1.45]],
 '_intermediates': [],
 '_inter_equations': [],
 '_equations': [<gekko.gekko.EquationObj at 0x1105f9700>],
 '_objectives': [],
 '_connections': [],
 '_objects': [],
 '_compounds': [],
 '_raw': [],
 'time': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
 '_model_initialized': True,
 '_csv_status': None,
 '_model': 'auto-generated',
 '_model_name': 'gk_model0',
 '_path': '/var/folders/s8/8ypwvww534g_q5s51__36bmc0000gp/T/tmp6zq07x1rgk_model0',
 'path': '/var/folders/s8/8ypwvww534g_q5s51__36bmc0000gp/T/tmp6zq07x1rgk_model0',
 '_extra_files': [],
 'solver_options': [],
 'csv_status': 'generated'}

In [24]:
#m.get_names()

Found x
Found dx


In [38]:
m.hello.name

'hello'

In [34]:
m.gk_logic_tree()

In [35]:
m.open_folder()

In [36]:
m.cleanup()

Directory /var/folders/s8/8ypwvww534g_q5s51__36bmc0000gp/T/tmp2vawc5vdheeey not found


In [37]:
m.clear()

In [55]:
dir(m)

['Array',
 'CV',
 'Connection',
 'Const',
 'Equation',
 'Equations',
 'FV',
 'GUI',
 'Intermediate',
 'MV',
 'Maximize',
 'Minimize',
 'Obj',
 'Param',
 'Raw',
 'SV',
 'Var',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_model',
 '_compounds',
 '_connections',
 '_constants',
 '_csv_status',
 '_equations',
 '_extra_files',
 '_generate_dbs_file',
 '_gui_open',
 '_id',
 '_ids',
 '_inter_equations',
 '_intermediates',
 '_model',
 '_model_initialized',
 '_model_name',
 '_objectives',
 '_objects',
 '_parameters',
 '_path',
 '_raw',
 '_remote',
 '_server',
 '_variables',
 '_write_csv',
 '_write_info',
 '_write_solver_options',
 'abs',
 'abs2',
 'abs3',
 'acos',
 'ac

In [28]:
m.x['test'] = m.SV(name = 'xtest')

In [30]:
m.x['test'].name

'xtest'

In [36]:
m.x

defaultdict(dict, {'test': 0})

In [35]:
print(m.test.name)
# dir(m.test)

test


In [46]:
import numpy as np
np.size(np.zeros((31,30)))

930